In [ ]:
import sys
sys.path.append('../qick/qick_lib/')
sys.path.append('../soft/')

from qick import *
from qick_training import *
from qick.averager_program import QickSweep
from qick.averager_program import QickRegister
from qick.averager_program import merge_sweeps

import numpy as np
from numpy.fft import fft, fftshift
import matplotlib.pyplot as plt

In [ ]:
# Load bitstream with custom overlay
soc = QickSoc('./qick_training.bit')
soccfg = soc

print(soccfg)

In [ ]:
class PhaseTest(NDAveragerProgram):
    def initialize(self):
        # Set the nyquist zone
        self.declare_gen(ch=self.cfg["gen_ch"][0], nqz=1)
        self.declare_gen(ch=self.cfg["gen_ch"][1], nqz=1)
        
        # Declare Gaussian Envelope.
        length = self.us2cycles(self.cfg['pulse_length'][0],gen_ch=self.cfg['gen_ch'][0])
        if self.cfg['pulse_style'] == 'gauss':
            self.add_gauss(ch=self.cfg['gen_ch'][0], name='env', sigma=length/2, length=length)
        else:
            length_ = length*self.soccfg['gens'][self.cfg['gen_ch'][0]]['samps_per_clk']
            xi = self.cfg['pulse_gain'][0]*np.ones(length_)
            self.add_pulse(ch=self.cfg['gen_ch'][0], name='env', qdata=xi)
        
        # First channel.
        freq = self.freq2reg(self.cfg['pulse_freq'][0], gen_ch=self.cfg['gen_ch'][0])
        self.default_pulse_registers(ch     = self.cfg['gen_ch'][0],  
                                     freq   = freq,
                                     phase  = 0,
                                     gain   = self.cfg['pulse_gain'][0])
        
        # Second channel.
        freq = self.freq2reg(self.cfg['pulse_freq'][1], gen_ch=self.cfg['gen_ch'][1])
        self.default_pulse_registers(ch     = self.cfg['gen_ch'][1], 
                                     style  = 'const', 
                                     freq   = freq,
                                     gain   = self.cfg['pulse_gain'][1], 
                                     length = self.us2cycles(self.cfg['pulse_length'][1],gen_ch=self.cfg['gen_ch'][1]))
                
        # Get frequency register.
        self.reg_f = self.get_gen_reg(self.cfg['gen_ch'][1], 'freq')
        
        # Define a QickRegister for the original value.
        self.reg_f_orig = QickRegister(self, self.reg_f.page, 20, name = "f_orig")
        self.reg_f_orig.set_to(self.reg_f)
        
        # Define a QickRegister for the jump.
        self.reg_f_jump = QickRegister(self, self.reg_f.page, 21, name = "f_jump")
        self.reg_f_jump.set_to(self.freq2reg(self.cfg['freq_jump']))
        
        self.synci(200)  # give processor some time to configure pulses
    
    def body(self):
        # Registers.
        self.set_pulse_registers(ch=self.cfg['gen_ch'][0], style='arb', waveform='env')
        self.set_pulse_registers(ch=self.cfg['gen_ch'][1], phase=0, phrst=0, mode="oneshot")
        
        # Trigger.
        self.trigger(pins=[0],t=40)

        # Pulses.
        self.pulse(ch = self.cfg['gen_ch'][0])
        self.pulse(ch = self.cfg['gen_ch'][1])
        
        # Wait, Jump and send Pulse.
        self.synci(self.us2cycles(self.cfg['wait']))        
        self.reg_f.set_to(self.reg_f_orig, "+", self.reg_f_jump)
        self.pulse(ch = self.cfg['gen_ch'][1])
        
        # Wait, Jump and send Pulse.
        self.synci(self.us2cycles(self.cfg['wait']))                
        self.reg_f.set_to(self.reg_f, "+", self.reg_f_jump)        
        self.pulse(ch = self.cfg['gen_ch'][1])
        
        # Wait, Jump and send Pulse.
        self.synci(self.us2cycles(self.cfg['wait']))                
        self.reg_f.set_to(self.reg_f_orig)        
        self.pulse(ch = self.cfg['gen_ch'][1])        
        
        # Period.
        self.synci(self.us2cycles(self.cfg['period']))
       
config={"gen_ch"      : [3,4],
        "reps"        : 5,
        "pulse_length": [2,0.3],
        "pulse_gain"  : [30000,20000], # [DAC units]
        "pulse_freq"  : [500,500], # [MHz]
        "pulse_style" : 'gauss',
        "wait"        : 0.2,
        "period"      : 5,
        "freq_jump"   : 30
       }

prog = PhaseTest(soccfg, config)
prog.config_all(soccfg)
soc.tproc.start()

In [ ]:
print(prog)

In [ ]:
# Plot Gaussian Envelope.
plt.figure(dpi=200)

# fs (MHz) and ts (us).
fs = soccfg['gens'][config['gen_ch'][0]]['fs']
ts = 1/fs

# Pulse data.
y = prog.pulses[config['gen_ch'][0]]['env']['data']
x = ts*np.arange(len(y))
plt.plot(x,y);
plt.xlabel('t [us]');
plt.legend(['I','Q']);